In [49]:
import sys

sys.setrecursionlimit(5000)

In [50]:
#!/usr/bin/env python
# coding: utf-8

from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.corpus import wordnet as wn

import numpy as np
import math
import nltk
import operator
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()



filename = 'wsj_raw_10000'

def dependency_parse(filename):
    ### This function takes in a file of sentences (each on a separate line)
    ### It runs a Stanford Dependency Parser (so assumes the user has downloaded 
    ### the Stanford CoreNLP parser and the English model)
    ### It returns a dictionary word_dict that maps a word W to another dictionary W_dict (=word_dict[W])
    ### W_dict stores all present triples containing the word W as the keys and their counts as the values.
    
    ### Please on Terminal run StanfordCoreNLPServer beforehand: (Using port 9000)
    ### java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer \
    ### -preload tokenize,ssplit,pos,lemma,ner,parse,depparse \
    ### -status_port 9000 -port 9000 -timeout 15000 & 
    
    # FOR TRIAL: process first N lines
    N = 100
    lincount = 0
    
    # Initialize a parser
    dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')
    
    # THIS LINE FOR MAC
    f = open(filename, "r") #, encoding="ISO-8859-1"
#     f = open(filename, 'r')
    word_dict = {} # word -> dict{triples that contain that word -> number}
    noun_dict = {} # noun -> number
    verb_dict = {} # verb -> number
    adj_dict = {} # adjective -> number
    rel_dict = {} # relationship -> number
    for line in f:
        # FOR TRIAL
#         if lincount >= N:
#             break
        
        line = line.strip('\n').strip()
        if len(line) <= 1 or line == '.START':
            # Meaningless lines in wsj_raw_all.pos
            continue
        # Parse the line/sentence and get the dependency triples
        parses = dep_parser.parse(line.split())
        triples = [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses][0]
        
        for triple in triples:
            w1 = str(triple[0][0]).lower()
            w1_pos = str(triple[0][1])
            r = str(triple[1])
            #jump the punctuation relations.
            if r == 'punct':
                continue
            w2 = str(triple[2][0]).lower()
            w2_pos = str(triple[2][1])
            # process the pos to something that is suitable for the huiyuan method
            w1_pos_ = w1_pos[0].lower()
            if w1_pos_ == 'n' or w1_pos_ == 'v' or w1_pos_ == 'j':
                if w1_pos_ == 'j':
                    w1_pos_ = 'a'
                w1 = lemmatizer.lemmatize(w1, w1_pos_)
                # generate noun, verb, adj dict
                if w1_pos_ == 'n':
                    if w1 in noun_dict:
                        noun_dict[w1] += 1
                    else:
                        noun_dict[w1] = 1
                elif w1_pos_ == 'v':
                    if w1 in verb_dict:
                        verb_dict[w1] += 1
                    else:
                        verb_dict[w1] = 1
                else:
                    if w1 in adj_dict:
                        adj_dict[w1] += 1
                    else:
                        adj_dict[w1] = 1
            w2_pos_ = w2_pos[0].lower()
            if w2_pos_ == 'n' or w2_pos_ == 'v' or w2_pos_ == 'j':
                if w2_pos_ == 'j':
                    w2_pos_ = 'a'
                w2 = lemmatizer.lemmatize(w2, w2_pos_)
                # generate noun, verb, adj dict
                if w2_pos_ == 'n':
                    if w2 in noun_dict:
                        noun_dict[w2] += 1
                    else:
                        noun_dict[w2] = 1
                elif w2_pos_ == 'v':
                    if w2 in verb_dict:
                        verb_dict[w2] += 1
                    else:
                        verb_dict[w2] = 1
                else:
                    if w2 in adj_dict:
                        adj_dict[w2] += 1
                    else:
                        adj_dict[w2] = 1
            # generate relation dict
            if r in rel_dict:
                #rel_dict[r].append((w1, r, w2))
                rel_dict[r] += 1
            else:
                #rel_dict[r] = [(w1, r, w2)]          
                rel_dict[r] = 1
            # Decide if word_dict has encounted the words w1 or w2
            # And if it has seen this dep triple in word_dict[w1] and word_dict[w2]
            if w1 in word_dict:
                if (w1, r, w2) in word_dict[w1]:
                    word_dict[w1][(w1, r, w2)] += 1
                else:
                    word_dict[w1][(w1, r, w2)] = 1
            else:
                word_dict[w1] = {}
                word_dict[w1][(w1, r, w2)] = 1
            if w2 in word_dict:
                if (w1, r, w2) in word_dict[w2]:
                    word_dict[w2][(w1, r, w2)] += 1
                else:
                    word_dict[w2][(w1, r, w2)] = 1
            else:
                word_dict[w2] = {}
                word_dict[w2][(w1, r, w2)] = 1
        
        # FOR TRIAL
#         print(lincount)
#         lincount += 1
#    for keys, values in rel_dict.items():
#        print(keys)
#        print(values)
    f.close()
    return word_dict, noun_dict, verb_dict, adj_dict, rel_dict

def print_word_dict(word_dict):
    ### Print out word_dict
    for word in word_dict:
        for triple in word_dict[word]:
            print(triple, end=', ')
            print(word_dict[word][triple])

def store_word_dict(word_dict):
    ### Write word_dict into a .txt file
    ### Each row is in the form:
    ### word    (word,r,word2)    count
    f = open('word_dict', 'w+')
    for word in word_dict:
        for triple in word_dict[word]:
            line = "%s\t(%s;%s;%s)\t%d\n" % (word, triple[0], triple[1], triple[2], word_dict[word][triple])
            f.write(line)
        f.write('\n')
    f.close()
    return True


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hanghang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hanghang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [51]:
#compute mutual infomation dictionary.
#Noun_dict can be replace by any dictionary with key = word, value = frequence of word.
#Verb_dict can be replaced by any dictionary with key = word, value = freq of word.
def Mutual_Info(word_dict, noun_dict, verb_dict, rel_dict):

    # This part computs maximum likelihood likelihood. In the logic of this code, MLE may not be needed to compute Mut_Inf
    # Compute maximum likelihood estimate for every pair of words with a specific relationship
    # rel_dict: dictionary storing relation --> frequency count

#     MLE_nouns = {}
#     count_triple = 0
#     count = 0

#     #count number of triples : ||*,*,*||
#     for i in word_dict.keys():
#         for j in word_dict[i].keys():
#             count_triple += word_dict[i][j]

#     #MLE: ||w,r,w'|| / ||*,*,*||
#     for noun in noun_dict.keys():
#         if noun not in word_dict().keys():
#             continue;
#         triples = word_list[noun]
#         MLE_nouns[noun] = {}
#         for triple in triples.keys():
#             MLE_nouns[noun][triple] = triples[triple] / count_triple

    #Mutual information: I(w,r,w') = log(||w,r,w'|| * ||*,r,*|| / (||w,r,*|| * ||*,r,w'||))
    Mut_Inf = {}
#     all_dict = {}
#     all_dict.update(noun_dict)
#     all_dict.update(verb_dict)
    all_words = noun_dict.keys() | verb_dict.keys()
    
    for word in all_words:
        # Initialize map for each word: its triple -> MLE prob
        Mut_Inf[word] = {}
        
    # Mut_Inf[w][(w,r,w')]
    for rel in rel_dict.keys():
        num = rel_dict[rel]     #||*,r,*||
        for word in all_words:            
            #compute ||w,r,*||
            count = 0
            for triple in word_dict[word].keys():
                #print(triple)
                if triple[0] == word and triple[1] == rel:
                    count += 1
            
            #record ||w,r,w'||*||*,r,*|| / ||w,r,*||
            if count == 0:
                continue
            for triple in word_dict[word].keys():
                if triple[1] == rel and triple[0] == word:          
                    Mut_Inf[word][triple] = word_dict[word][triple] * num / count
                        
    #compute "||*,r,w'||"
    R_W = {}
    for rel in rel_dict.keys():
        for word in word_dict.keys():
            for triple in word_dict[word].keys():
                if triple[1] == rel and triple[2] == word:
                    t = (rel,word)
                    if t in R_W.keys():
                        R_W[t] += 1
                    else:
                        R_W[t] = 1

    #compute Mut_Inf
    #match ||*,r,w'|| with entries in Mut_Inf 
    for pair in R_W.keys():
        for word in Mut_Inf.keys():
            for triple in Mut_Inf[word].keys():
                if (triple[1],triple[2]) == pair:
                    num = Mut_Inf[word][triple] / R_W[pair]
                    if num > 0:
                        Mut_Inf[word][triple] = math.log(num)
                    else:
                        Mut_Inf[word][triple] = 0

    return Mut_Inf

In [52]:
#Compute T_W for every w in noun_dict. Note Mut_Inf should be compatible with noun_dict
#T(W) is a dictionary with key = word (in noun_dict), value = T(word) a set of pairs (relation, word2)
def T_W(noun_dict,Mut_Inf):
    T_w = {}
    for word in noun_dict.keys():
        #compute T(w1)=T(word1)
        T_word = set()
        for triple in Mut_Inf[word].keys():
            if Mut_Inf[word][triple] > 0:
                T_word.add((triple[1],triple[2]))
        T_w[word] = T_word
    return T_w

In [53]:
#Compute similarity matrix for each pair of words in noun_dict. 
#Noun_dict can be replace by any dictionary with key = word, value = frequence of word.
#Verb_dict can be replaced by any dictionary with key = word, value = freq of word. It is used as the Iterative Sim.
# INPUT:
# 1. Mut_Inf: Mutual information dictionary
# 2. noun_dict: dictionary of nouns, or sim(w1,w2)
# 3. verb_dict: dictionary of verbs or sim(w1',w2')
# 4. rel_dict: dictionary of relationships
# 5. T_w: dictionary of T(w)
# 6. sim0: initial similarity, (empty dictionary in our case)
# RETURN:
# a similarity dictionary
def sim_dict(Mut_Inf, noun_dict, verb_dict, rel_dict, T_w, sim0):
                
#     all_dict = {}
#     all_dict.update(noun_dict)
#     all_dict.update(verb_dict)
        
    # compute similarity dictionary for verb_dict using cosine-similarity BEFORE THE ITERATION
#     if sim0_flag == 0:
#         sim0 = sim_zero(T_w, verb_dict)
#     # in iteration, directly let sim0 = sim
#     else:
#         sim0 = verb_dict
        

    #compute similarity dictionary, sim(word1, word2)
    sim = {}
    for word in noun_dict.keys():
        sim[word] = {}
    
    for word1 in noun_dict.keys():
        for word2 in noun_dict.keys():
            
            if word1 == word2:
                continue
                
            #use symmetry
            if word2 in sim.keys() and word1 in sim[word2].keys():
                sim[word1][word2] = sim[word2][word1]
                continue
             
            intersection = T_w[word1] & T_w[word2]
            
            #Sum1 is the first term in the numerator
            Sum1 = 0
            for pair in intersection:
                r,w = pair
                #print(word1,r,w)
                Sum1 += Mut_Inf[word1][(word1,r,w)] + Mut_Inf[word2][(word2,r,w)]
                
            #Sum2 is the second term in the numerator
            Sum2 = 0
            for pair1 in T_w[word1]:
                if pair1[1] in verb_dict.keys():                                 #(r,w1') in T(w1)
                    rel = pair1[0]
                    verb1 = pair1[1]
                    for pair2 in T_w[word2]:
                        if pair2[1] in verb_dict.keys() and pair2[0] == rel:     #(r,w1') in T(w1) and (r,w2') in T(w2)
                            verb2 = pair2[1]  
                            if verb1 != verb2:                                   #not allow verb1 = verb2
                                if verb1 in sim0.keys() and verb2 in sim0[verb1].keys():
                                    Sum2 += sim0[verb1][verb2] * (Mut_Inf[word1][(word1,rel,verb1)] + Mut_Inf[word2][(word2,rel,verb2)])
                
            
            #Sum3 is the first two terms in the denominator
            Sum3 = 0
            for pair in T_w[word1]:
                r,w = pair
                Sum3 += Mut_Inf[word1][(word1,r,w)]
            for pair in T_w[word2]:
                r,w = pair
                Sum3 += Mut_Inf[word2][(word2,r,w)]
                
            #altogether
            #only valid if denominator is none zero(so we have proper context), and record only if similarity is nonzero
            if (Sum2 != 0 or Sum3 != 0):
                if (Sum1 != 0 or Sum2 != 0):
                    if word1 in sim.keys():
                        sim[word1][word2] = (Sum1 + Sum2) / (Sum2 + Sum3)
                    else:
                        sim[word1] = {}
                        sim[word1][word2] = (Sum1 + Sum2) / (Sum2 + Sum3) 

    return sim
            

In [54]:
# #compute cosine similarity between the context information given (parameter: verb_dict)
# def sim_zero(T_w, verb_dict):
#     sim0 = {}
#     for verb1 in verb_dict.keys():
#         sim0[verb1] = {}
#         for verb2 in verb_dict.keys():
#             #we don't want sim0(v,v)
            
#             if verb1 == verb2:
#                 continue
            
#             #use symmetry
#             if verb2 in sim0.keys() and verb1 in sim0[verb2].keys():
#                 sim0[verb1][verb2] = sim0[verb2][verb1]
#                 continue
            
#             if len(T_w[verb1]) == 0 or len(T_w[verb2]) == 0:
#                 continue
            
#             intersection = T_w[verb1] & T_w[verb2]
#             sim0[verb1][verb2] = len(intersection) / ((len(T_w[verb1]) * len(T_w[verb2]))) ** 0.5
#     return sim0

In [55]:
# sort the similarity matrix of a given word, from high to low
# INPUT:
# 1. sim: similarity dictionary
# 2. a word, as a key to sim, whose similarity to other words is to be sorted
# RETURN:
# a list of tuples (word2,similarity)
def sort_sim(sim, word):
    if word not in sim.keys():
        return {}
    sim_sort = sorted(sim[word].items(), key=operator.itemgetter(1), reverse = True)
    return sim_sort
    

In [56]:
word_dict, noun_dict, verb_dict, adj_dict, rel_dict = dependency_parse(filename)
store_word_dict(word_dict)
#print_word_dict(word_dict)  

True

In [57]:
#This part is for running on a computing machine


#write dicts
#for word_dict, change name of file in the method
store_word_dict(word_dict)

f = open('noun_dict', 'w+')
for key,value in noun_dict.items():
    line = key + "\t" + str(value) + "\n"
    f.write(line)
f.close()

f = open('verb_dict', 'w+')
for key,value in verb_dict.items():
    line = key + "\t" + str(value) + "\n"
    f.write(line)
f.close()

f = open('adj_dict', 'w+')
for key,value in adj_dict.items():
    line = key + "\t" + str(value) + "\n"
    f.write(line)
f.close()

f = open('rel_dict', 'w+')
for key,value in rel_dict.items():
    line = key + "\t" + str(value) + "\n"
    f.write(line)
f.close()
print("write dicts complete")

# word_dict = {}
# file = open('word_dict','r') 
# content = file.readlines()
# for i in range(len(content)):
#     line = content[i].strip()
#     if len(line) == 0:
#         continue
#     line = line.split('\t')
#     word = line[0]
#     triple = tuple(map(str,line[1].strip('()').split(';')))
#     count = int(line[2])
#     if word in word_dict.keys():
#         word_dict[word][triple] = count
#     else:
#         word_dict[word] = {}
#         word_dict[word][triple] = count
# file.close()
# #print(word_dict == word_dict1)


# noun_dict = {}
# file = open('noun_dict','r') #,encoding = 'iso-8859-1'
# content = file.readlines()
# # print(content)
# for i in range(len(content)):
#     line = content[i].strip().split("\t")
#     word = line[0]
#     count = int(line[1])
#     noun_dict[word] = count
# file.close()
# # print(noun_dict1 == noun_dict)

# verb_dict = {}
# file = open('verb_dict','r')
# content = file.readlines()
# for i in range(len(content)):
#     line = content[i].strip().split("\t")
#     word = line[0]
#     count = int(line[1])
#     verb_dict[word] = count
# file.close()
# # print(verb_dict1 == verb_dict)

# adj_dict = {}
# file = open('adj_dict','r')
# content = file.readlines()
# for i in range(len(content)):
#     line = content[i].strip().split("\t")
#     word = line[0]
#     count = int(line[1])
#     adj_dict[word] = count
# file.close()
# # print(adj_dict1 == adj_dict)

# rel_dict = {}
# file = open('rel_dict','r')
# content = file.readlines()
# for i in range(len(content)):
#     line = content[i].strip().split("\t")
#     word = line[0]
#     count = int(line[1])
#     rel_dict[word] = count
# file.close()
# # print(rel_dict1 == rel_dict)

write dicts complete


In [58]:
Mut_Inf = {}
#sim = verb_dict
useful_dict = {}
useful_dict.update(noun_dict)
useful_dict.update(verb_dict)
#useful_dict.update(adj_dict)
#extra_info_dict = verb_dict
# sim0 = recover_sim('sim_result_part_iter1')
Mut_Inf = Mutual_Info(word_dict,noun_dict,verb_dict,rel_dict)
T_w = T_W(useful_dict,Mut_Inf)
#sim0_flag = 0
#sim0 = sim_zero(T_w,useful_dict)

#try iteration
sim0 = {}
for i in range(20):
    print("start of " + str(i+1) + "th iteration")
    sim  = sim_dict(Mut_Inf,noun_dict,verb_dict,rel_dict,T_w,sim0)
    file = open("sim_result_10000_iter" + str(i+1),"w")
    for word1 in sim.keys():
        sim_sort = sort_sim(sim,word1)
        #print(sim_sort)
        if len(sim_sort) > 100:
            for j in range(100):
                word2 = sim_sort[j][0]
                similarity = sim_sort[j][1]
                file.write(word1 +'\t'+ word2 +'\t'+ str(similarity) + '\n')
        else:
            for j in range(len(sim_sort)):
                word2 = sim_sort[j][0]
                similarity = sim_sort[j][1]
                file.write(word1 +'\t'+ word2 +'\t'+ str(similarity) + '\n')        
    file.close()
    sim0 = sim
    if i == 18:
        sim1 = sim
    if i == 19:
        sim2 = sim

count1 = 0
count2 = 0
count3 = 0
for k1 in sim1.keys():
    for k2 in sim1[k1].keys():
        if k2 in sim2[k1].keys() and k2 in sim2[k1].keys():
            if sim1[k1][k2] < sim2[k1][k2]:
                count1 += 1
            if sim1[k1][k2] > sim2[k1][k2]:
                count2 += 1    
            if sim1[k1][k2] == sim2[k1][k2]:
                count3 += 1

print(count1)
print(count2)
print(count3)

start of 1th iteration
start of 2th iteration
start of 3th iteration
start of 4th iteration
start of 5th iteration
start of 6th iteration
start of 7th iteration
start of 8th iteration
start of 9th iteration
start of 10th iteration
start of 11th iteration
start of 12th iteration
start of 13th iteration
start of 14th iteration
start of 15th iteration
start of 16th iteration
start of 17th iteration
start of 18th iteration
start of 19th iteration
start of 20th iteration
1086904
142
7134024


In [59]:
# Evaluation based on synsets in WordNet
# Given a word, POS w_pos and a sim, compute the average sim between any two pairs in synsets of the word
# w_pos takes "n", "v", "a" for now
def synset_measure(w, w_pos, sim):
    # INPUT: 1. (string) word w
    #        2. (string) "n" or "v" or "a"
    #        3. a similarity dict
    # RETURN: sum of all similarities, count of all compared pairs
    if w_pos == "n":
        w_pos = wordnet.NOUN
    if w_pos == "v":
        w_pos = wordnet.VERB
    if w_pos == "a":
        w_pos = wordnet.ADJ
    synsets = wordnet.synsets(w, pos=w_pos)
    sim_total = 0
    pair_count = 0
    for synset in synsets:
        syn_words = [w]
        for lemma in synset.lemmas():
            w2 = lemma.name()
            if (w2 in syn_words) or (not w2 in sim): 
                # Words like "domestic_dog" may not be in sim_dict, and we skip such words
                continue
            for w1 in syn_words:
                sim_total += sim(w1, w2)
                pair_count += 1
            syn_words.append(w2)
    return sim_total, pair_count

In [60]:
# sim_wn(w1, w2) = max_{c1 in S(w1) and c2 in S(w2)} ( max{c in super(c1) intersects super(c2)} (2log(P(c))) / (log(P(c1))+log(P(c2))))
# noun_dict: dictionary
# verb_dict: dictionary
# output: sim_wn, dictionary, keys: (w1, w2), values: sim

def sim_wn(noun_dict, verb_dict):  
    useful_dict = {} # Merge two dict
    for key in (noun_dict.keys() | verb_dict.keys()):
        if key in noun_dict.keys():
            useful_dict[key] = noun_dict[key]
        if key in verb_dict.keys():
            if key in useful_dict.keys():
                useful_dict[key] += verb_dict[key]
            else:
                useful_dict[key] = verb_dict[key]

    sim_wn_dict = {}
    all_words = list(useful_dict.keys())
    L = len(all_words) # To avoid repeated comparisons
    superclass_dict = {} # To avoid repeated super_class() for same synset; store: synset.name() --> synset superclasses
    P_dict = {} # To avoid repeated getP(); store: synset.name() --> P(synset)
    k = 1
    for i in range(L):  
        w1 = all_words[i]
        for j in range(i+1,L):  
            w2 = all_words[j]
            if w1 == w2:
                continue                
            else:                
                # first max
                max1 = 0 # Lowest similarity                
                S_w1 = wn.synsets(w1) # S(w1)
                S_w2 = wn.synsets(w2) # S(w2)
                
                target_dict = {} # make sure which dict to use while computing P(*)
                if w1 in noun_dict and w2 in noun_dict:
                    target_dict = noun_dict
                elif w1 in verb_dict and w2 in verb_dict:
                    target_dict = verb_dict
                else:
                    target_dict = useful_dict
                for c1 in S_w1:                    
                    for c2 in S_w2:
                        # second max
                        max2 = 0 # Lowest similarity
                        c1_name = c1.name()
                        c2_name = c2.name()
                                
                        if c1_name in P_dict:
                                P_c1 = P_dict[c1_name]
                        else:
                            P_c1 = get_P(c1, target_dict, P_dict) # P(c1)
                        if c2_name in P_dict:
                            P_c2 = P_dict[c2_name]
                        else:
                            P_c2 = get_P(c2, target_dict, P_dict) # P(c2)
                        if P_c1 == 0 or P_c2 == 0:
                                continue
                        
                        if c1_name in superclass_dict:
                            super_c1 = superclass_dict[c1_name]
                        else:
                            super_c1 = super_class(c1, superclass_dict)
                        if c2_name in superclass_dict:
                            super_c2 = superclass_dict[c2_name]
                        else:
                            super_c2 = super_class(c2, superclass_dict)
                            
                        intersection = set(super_c1) & set(super_c2) 
                        for c in intersection:
                            if c.name() in P_dict:
                                P_c = P_dict[c.name()]
                            else:
                                P_c = get_P(c, target_dict, P_dict) # P(c)
        
                            if P_c == 0:
                                continue
                            curr_max2 = (2 * math.log(P_c)) / (math.log(P_c1) + math.log(P_c2))
#                             print(curr_max2)
                            if curr_max2 > max2:
                                max2 = curr_max2
                        if max2 > max1:
                            max1 = max2  
                # store the value as sim_wn(w1, w2)
                if max1 > 0:
                    if w1 in sim_wn_dict.keys():
                        sim_wn_dict[w1][w2] = max1
                    else:
                        sim_wn_dict[w1] = {}
                        sim_wn_dict[w1][w2] = max1
#         print(k)
        k += 1
    print(P_dict)
    
    return sim_wn_dict

In [61]:
# find all super classes of a SYNSET
# ss: synset
# sc_dict: dictionary; key: synset.name, value: super-synset list

# ss = wn.synset('dog.n.01')
def super_class(ss, sc_dict):
    if ss.name() in sc_dict:
        # Save time if it has been investigated
        return sc_dict[ss.name()]
    cur_super = ss.hypernyms()
    cur_all = [] # All super-synsets of given ss
    if cur_super != []:
        for super_ss in cur_super:
            cur_all.append(super_ss)
            cur_all = list(set(cur_all + super_class(super_ss, sc_dict))) # Combine and remove duplicates if any
    sc_dict[ss.name()] = cur_all
    return cur_all
#                 print(ss)

# xx = super_class(cc)
# print(cc.hypernyms())

In [62]:
# Helper function for get_P()
# INPUT:
# 1. ss: WordNet synset
# 2. xx_dict: (dictionary) key: word, value: its frequency count
# 3. word_seen: words that have been counted already
# 4. synset_seen: synsets that have been visited already
# 5. C_dict: (dictionary of synsets that have finished counting for DP) key: synset.name(), value: its instance count C
# RETURN:
# instance count C for the given synset ss
def get_subclass_count(ss, xx_dict, word_seen, synset_seen, C_dict):
#     print(ss.name())
    if ss.name() in C_dict:
        return C_dict[ss.name()]
    if ss.name() in synset_seen:
        # has entered this synset already
        return 0
    # Base case: count words under current synset
    C = 0 # P(ss)
    for lemmas in ss.lemmas():
        cur_word = lemmas.name()
        if (cur_word in xx_dict) and (not cur_word in word_seen):
            C += xx_dict[cur_word]
            word_seen.append(cur_word)
    synset_seen.append(ss.name())
    
    # Recursion: loop over the sub-synsets of current synset
    cur_sub = ss.hyponyms()
    if cur_sub != []:
        for sub_ss in cur_sub:
            C += get_subclass_count(sub_ss, xx_dict, word_seen, synset_seen, C_dict)
    C_dict[ss.name()] = C
    return C


# calculate P(*)
# nominator: the frequency of all words in synset
# denominator: all nouns / verbs / all nouns + verbs, depend on the pos of w1, w1
# ss: synset
# xx_dict: dictionary, key: word, values: frequency
# P_dict: dictionary, key: synset.name, value: P(synset)
# output: P(*)

def get_P(ss, xx_dict, P_dict):
    # Avoid repeated count in downward search of words
    word_seen = []
    # Avoid synset cycles
    synset_seen = []
    # to get words in synset, use synset.lemmas().name() staff
    C_dict = {} # dictionary: synset.name --> synset count
    nominator = get_subclass_count(ss, xx_dict, word_seen, synset_seen, C_dict)
    
    # denominator
    denominator = 0
    for keys in xx_dict.keys():
        denominator += xx_dict[keys]
    
    PP = nominator / denominator
    
    for synset in C_dict:
        if not synset in P_dict:
            P_dict[synset] = C_dict[synset] / denominator
#     if PP == 0:
#         print('yeah')
    
    return PP

# ss = wn.synsets('nation')[0]
# P_dict = {}
# get_P(ss, noun_dict, P_dict)


In [63]:
# Evaluation by Lin's method (1998)
# INPUT:
# 1. dict1: key: word, value: dict: word' --> sim(word, word')
# 2. dict2: same
# 3. N: number of top N similar words to compare
# RETURN:
# cos_dict: shared key word --> cosine similarity for it

def compare_thesaurus(dict1, dict2, N):
    cos_dict = {}
    for word in (set(dict1.keys()) & set(dict2.keys())):
        vec1 = sort_sim(dict1, word) # List of tuples (word', sim)
        vec2 = sort_sim(dict2, word)
        cos_dict[word] = cos_sim(vec1, vec2, N)
    return cos_dict

# Helper function for computing cosine sim
# INPUT:
# 1. vec1: list of tuples (word, sim)
# 2. vec2: same
# 3. N: largest vector length
# RETURN:
# cosine similarity between vec1 and vec2
def cos_sim(vec1, vec2, N):
    if len(vec1) > N:
        vec1 = vec1[0:N] #Truncate
    if len(vec2) > N:
        vec2 = vec2[0:N]
    # For denominaor
    sim_vec1 = [tup[1] for tup in vec1]
    sim_vec2 = [tup[1] for tup in vec2]
    vec1_norm = np.linalg.norm(sim_vec1)
    vec2_norm = np.linalg.norm(sim_vec2)
    # For nominator
    nominator = 0
    dict_vec1 = {} # Transform into dict word' --> sim
    dict_vec2 = {}
    for tup in vec1:
        dict_vec1[tup[0]] = tup[1]
    for tup in vec2:
        dict_vec2[tup[0]] = tup[1]
    for w in (dict_vec1.keys() & dict_vec2.keys()):
        nominator += dict_vec1[w] * dict_vec2[w]
    return nominator / (vec1_norm * vec2_norm)

In [64]:
# Helper function for recovering similarity dictionary
# INPUT:
# 1. file_name : a string representing file name
# RETURN:
# a dictionary of similarity, in the form: (word1:{(word2:similarity)...}), a dictionary of dictionary. 
# first level dict: word1 -> {}
# second leve dict: word2 -> similarity of w1 and w2
def recover_sim(file_name):
    sim = {}
    file = open(file_name,'r') #,encoding = 'iso-8859-1'
    content = file.readlines()
    for line in content:
        line = line.strip()
        if len(line) == 0:
            continue
        splitted = line.split('\t')
        word1 = splitted[0]
        word2 = splitted[1]
        similarity = float(splitted[2])
        if word1 in sim.keys():
            sim[word1][word2] = similarity
        else:
            sim[word1] = {}
            sim[word1][word2] = similarity
    return sim

# method for reading a dict from a file
# INPUT:
# 1:file_name: name of file
# RETURN: a dictionary from file, key = word, value = count
def read_dict(file_name):
    any_dict = {}
    file = open(file_name,'r') #,encoding = 'iso-8859-1'
    content = file.readlines()
    # print(content)
    for i in range(len(content)):
        line = content[i].strip().split("\t")
        word = line[0]
        count = int(line[1])
        any_dict[word] = count
    file.close()
    return any_dict

In [65]:
# test sim_wn

#word_dict, noun_dict, verb_dict, adj_dict, rel_dict = dependency_parse(filename)
sim_wn_dict = sim_wn(noun_dict, verb_dict)

file_wn = open('wordnet_sim_raw.txt', 'w')
for keys, values in sim_wn_dict.items():
    if value != 0:
        file_wn.write(keys[0])
        file_wn.write('\t')
        file_wn.write(keys[1])
        file_wn.write('----->')
        file_wn.write(str(values))
        file_wn.write('\n')
file_wn.close()
file_wn2 = open('wordnet_sim_raw_makeSense.txt', 'w')
for keys, values in sim_wn_dict.items():
    if values != -1:
        file_wn2.write(keys[0])
        file_wn2.write('\t')
        file_wn2.write(keys[1])
        file_wn2.write('----->')
        file_wn2.write(str(values))
        file_wn2.write('\n')
file_wn2.close()
# for keys, values in sim_wn_dict.items():
#     print(keys)
#     print(values)

{'clove.n.03': 0.0, 'garlic.n.02': 1.0523269579858462e-05, 'country_doctor.n.01': 0.0, 'family_doctor.n.01': 0.0, 'general_practitioner.n.01': 0.0, 'ail.v.01': 1.0523269579858462e-05, 'erupt.v.08': 1.5784904369787693e-05, 'trouble.v.05': 0.0003051748178158954, 'upjohn.n.01': 0.0, 'crouch.n.01': 3.136418523687801e-05, 'ballast.n.03': 0.0, 'unit_character.n.01': 0.0, 'character.n.09': 0.0, 'good-temperedness.n.01': 0.0, 'cheerfulness.n.01': 7.841046309219502e-05, 'common_denominator.n.02': 0.0, 'depth.n.06': 0.0, 'eidos.n.01': 0.0, 'ethos.n.01': 0.0, 'human_nature.n.01': 0.0, 'background.n.01': 0.0, 'birthright.n.01': 0.0, 'birthright.n.03': 0.0, 'education.n.05': 0.0, 'raising.n.02': 7.841046309219502e-05, 'upbringing.n.01': 7.841046309219502e-05, 'inheritance.n.04': 7.841046309219502e-05, 'anal_personality.n.01': 0.0, 'genital_personality.n.01': 0.0, 'gender_identity.n.01': 0.0, 'identification.n.05': 0.0, 'personhood.n.01': 0.0, 'identity.n.01': 0.0, 'narcissistic_personality.n.01': 0

IndexError: string index out of range

In [74]:
sim1 = recover_sim('sim_result_10000_iter1')
sim5 = recover_sim('sim_result_10000_iter5')
sim10 = recover_sim('sim_result_10000_iter10')
print(sim1 == sim20)

False


In [66]:
print(len(sim_wn_dict))

3086


In [80]:
for i in range(20):
    sim = recover_sim('sim_result_10000_iter' + str(i+1))
    re = compare_thesaurus(sim,sim_wn_dict,100)
    Sum = 0
    for value in re.values():
        Sum += value
    print('iter ' + str(i+1) + ' of 20: ' + str(Sum/len(re)))
   


iter 1 of 20: 0.018953149630575986
iter 2 of 20: 0.020378508309906115
iter 3 of 20: 0.020583381498826034
iter 4 of 20: 0.02062454294707912
iter 5 of 20: 0.020630286749300116
iter 6 of 20: 0.02064214902718897
iter 7 of 20: 0.0206356120734704
iter 8 of 20: 0.020635621867417662
iter 9 of 20: 0.02063562375366163
iter 10 of 20: 0.020635624117043352
iter 11 of 20: 0.02063562418705119
iter 12 of 20: 0.020635624200538886
iter 13 of 20: 0.020635624203137387
iter 14 of 20: 0.02063562420363805
iter 15 of 20: 0.020635624203734514
iter 16 of 20: 0.020635624203753086
iter 17 of 20: 0.020635624203756676
iter 18 of 20: 0.02063562420375735
iter 19 of 20: 0.020635624203757506
iter 20 of 20: 0.020635624203757533


In [ ]:
.020647720114779666
.020647720028014658
.020642379531685403